In [ ]:
from __future__ import print_function

"""
A script to run the cqed_rhf and cqed_cis method on MgH+ potential energy surface in a cc-pVDZ basis set,
reproducing data from Figure 3 by McTague and Foley.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cis import *
from helper_cs_cqed_cis import *
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
from matplotlib import pyplot as plt
# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


Methods to fit Pauli-Fierz and Jaynes-Cummings models to CIS data for the ground- and excited-state
of a quantum system (including transition dipole moments in both cases, and the total dipole
moments in the Pauli-Fierz case) given a photon frequency and coupling strength.

In [ ]:
# functions for Pauli-Fierz and Jaynes-Cummings model Hamiltonians

def pauli_fierz(e1, e2, omega, lam, mu_g, mu_e, mu_eg):
    H = np.zeros((3,3),dtype=complex)
    l_d_mu_g = np.dot(lam, mu_g)
    l_d_mu_e = np.dot(lam, mu_e)
    l_d_tdm  = np.dot(lam, mu_eg)
    
    H[0,0] = e1 + 0.5 * l_d_mu_g ** 2
    H[1,1] = e1 + omega + 0.5 * l_d_mu_g ** 2
    H[2,2] = e2 + 0.5 * l_d_mu_e ** 2
    H[1,2] = np.sqrt(omega/2) * l_d_tdm
    H[2,1] = np.sqrt(omega/2) * l_d_tdm

    
    # diagonalize
    vals, vecs = la.eig(H)
    idx = vals.argsort()[::1]
    vals = vals[idx]
    vecs = vecs[:,idx]
    return vals[0], vals[1], vals[2], vecs

def jaynes_cummings(e1, e2, omega, lam, mu_eg):
    H = np.zeros((3,3),dtype=complex)
    l_d_tdm = np.dot(lam, mu_eg)
    H[0,0] = e1 + omega
    H[1,1] = e2
    H[0,1] = np.sqrt(omega/2) * l_d_tdm
    H[1,0] = np.sqrt(omega/2) * l_d_tdm
    
    # diagonalize
    vals, vecs = la.eig(H)
    idx = vals.argsort()[::1]
    vals = vals[idx]
    return vals[0], vals[1]


# MgH+ ground-state surfaces with strong coupling (Fig 5)

This cell will compute the ground-state potential energy surface with RHF, QED-RHF, and QED-CIS to determine if strong-coupling alters the equilibrium geometry of the ground-state

In [ ]:
# specific MgH+ geometry
mol_str = """
Mg
H 1 1.3
symmetry c1
1 1
"""

# template for z-matrix with variable bondlength
mol_tmpl = """
Mg
H 1 **R**
symmetry c1
1 1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk'}

# set psi4 options and geometry
psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)

# set psi4 options
psi4.set_options(options_dict)

# array of bondlengths for MgH+
r_array = np.array([1.60, 1.61, 1.62, 1.63, 1.64, 1.65, 1.66, 1.67, 1.68, 1.69, 1.70]) #, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3])



# array to hold the cs-cqed-cis energies
# array for cqed cis
cqed_cis_energy_array = np.zeros(len(r_array),dtype=complex)
cqed_rhf_energy_array = np.zeros(len(r_array))
rhf_energy_array = np.zeros(len(r_array))
# electric field strength 
lam_z = 0.0125

# photon energy
om_r = 4.75 / 27.211

# approximate maximum value of transition dipole moment in a.u.
max_tdm = 2.25

# approximate max value of g
g_max = np.sqrt(om_r/2) * lam_z * max_tdm

print(" value of gamma to cross over from weak to strong coupling:", 4 * g_max)
om = om_r - 0 * (g_max/2) * 1j

# lambda vector
lam = np.array([0,0,lam_z])

# loop over the different bond-lengths, create different instances
# of MgH+ molecule and compute and store various quantities
ctr = 0
for r in r_array:
    molstr = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(molstr)
    e, wfn = psi4.energy("scf/cc-pVDZ", return_wfn=True, molecule=mol)
    cqed_dict = cs_cqed_cis(lam, om, molstr, options_dict)
    
    # get cqed-rhf energy
    cqed_scf_e = cqed_dict['CQED-RHF ENERGY']
    # get cqed-cis excitation energies
    cqed_cis_e = cqed_dict['CQED-CIS ENERGY']
    # store to arrays
    cqed_cis_energy_array[ctr] = cqed_scf_e+cqed_cis_e[0]
    rhf_energy_array[ctr] = e
    cqed_rhf_energy_array[ctr] = cqed_scf_e
    ctr+=1
    

 

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
from scipy.interpolate import InterpolatedUnivariateSpline

# fit each surface to a spline
cqed_cis_spline = InterpolatedUnivariateSpline(r_array, np.real(cqed_cis_energy_array), k=3)
cqed_rhf_spline = InterpolatedUnivariateSpline(r_array, np.real(cqed_rhf_energy_array), k=3)
rhf_spline = InterpolatedUnivariateSpline(r_array, np.real(rhf_energy_array), k=3)

# evaluate each spline on a finer grid
r_fine = np.linspace(1.6, 1.7, 100)
cqed_cis_e_fine = cqed_cis_spline(r_fine)
cqed_rhf_e_fine = cqed_rhf_spline(r_fine)
rhf_e_fine = rhf_spline(r_fine)

# get curvatures of each surface
cqed_cis_f = cqed_cis_spline.derivative()
cqed_cis_c = cqed_cis_f.derivative()

cqed_rhf_f = cqed_rhf_spline.derivative()
cqed_rhf_c = cqed_rhf_f.derivative()

rhf_f = rhf_spline.derivative()
rhf_c = rhf_f.derivative()

# get locations of equilibrium geometry
r_eq_cqed_cis = r_fine[np.argmin(cqed_cis_e_fine)]
r_eq_cqed_rhf = r_fine[np.argmin(cqed_rhf_e_fine)]
r_eq_rhf = r_fine[np.argmin(rhf_e_fine)]

# print location of equilibrium bondlength
print("r_eq from cqed_cis",r_eq_cqed_cis)
print("r_eq from cqed_rhf",r_eq_cqed_rhf)
print("r_eq from rhf",r_eq_rhf)

# print curvature evaluated at equilibrium geometries
k_cqed_cis = cqed_cis_c(r_eq_cqed_cis)
k_cqed_rhf = cqed_rhf_c(r_eq_cqed_rhf)
k_rhf = rhf_c(r_eq_rhf)

# print the force constant of each surface in atomic units
print("force constant from cqed_cis",k_cqed_cis)
print("force constant from cqed_rhf",k_cqed_rhf)
print("force constant from rhf",k_rhf)


# plot surfaces on fine grid
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 12
plt.plot(r_fine, cqed_cis_e_fine, 'red')
plt.plot(r_fine, cqed_rhf_e_fine, 'blue')
plt.plot(r_fine, rhf_e_fine, 'green')
plt.xlabel("Bondlength (Angstroms)")
plt.ylabel("Energy (Hartrees)")
plt.legend()
plt.show()

In [ ]:
# print values to file for gnuplotting
for i in range(0, len(r_array)):
    print(r_array[i], rhf_energy_array[i], np.real(cqed_rhf_energy_array[i]), np.real(cqed_cis_energy_array[i]))




# Equilibrium bondlength of excited and polaritonic surfaces
This will explore the equilibrium bondlength of the first few excited states from CIS and the lower- and upper-polariton surfaces from QED-CIS.  

In [ ]:
# template for z-matrix
mol_tmpl = """
Mg
H 1 **R**
symmetry c1
1 1
"""

# set psi4 options
psi4.set_options(options_dict)
# array of bondlengths for MgH+
r_array = np.array([1.85,1.86,1.87,1.88,1.89,1.90, 1.91, 1.92, 1.93, 1.94, 1.95, 1.96, 1.97, 1.98]) #, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3])

# arrays to hold psi4 cis quantities to fit the model PF Hamiltonian with
# array for transition dipole moments
tdm_array = np.zeros((3, len(r_array)))

# array for total diplole of ground and excited state 
dipole_array = np.zeros((3,2,len(r_array)))

# array for ordinary cis
cis_energy_array = np.zeros((5,len(r_array)))

# array to hold the cs-cqed-cis energies
# array for cqed cis
cqed_cis_energy_array = np.zeros((5,len(r_array)),dtype=complex)

# array for model Pauli-Fierz energies
pf = np.zeros((3,len(r_array)),dtype=complex)

# array for Jaynes-Cummings energies
jc = np.zeros((2,len(r_array)),dtype=complex)

# electric field strength 
lam_z = 0.0125

# photon energy
om_r = 4.75 / 27.211

# approximate maximum value of transition dipole moment in a.u.
max_tdm = 2.25

# approximate max value of g
g_max = np.sqrt(om_r/2) * lam_z * max_tdm

print(" value of gamma to cross over from weak to strong coupling:", 4 * g_max)
om = om_r 

# lambda vector
lam = np.array([0,0,lam_z])

# loop over the different bond-lengths, create different instances
# of MgH+ molecule and compute and store various quantities
ctr = 0
for r in r_array:
    molstr = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(molstr)
    e, wfn = psi4.energy("scf/cc-pVDZ", return_wfn=True, molecule=mol)
    
    # confirm our cis agrees with psi4
    res = tdscf_excitations(wfn, states=3, triplets = "NONE", tda=True)
    
    # parse res for excitation energies
    psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]
    
    cis_energy_array[0,ctr] = e
    cis_energy_array[0,ctr] = e+psi4_excitation_e[0]
    
    
    cqed_dict = cs_cqed_cis(lam, om, molstr, options_dict)
    
    # get cqed-rhf energy
    cqed_scf_e = cqed_dict['CQED-RHF ENERGY']
    # get cqed-cis excitation energies
    cqed_cis_e = cqed_dict['CQED-CIS ENERGY']
    # store to arrays
    cqed_cis_energy_array[0:5,ctr] = cqed_scf_e+cqed_cis_e[0:5]
    ctr+=1
    

 

Plot the polaritonic surfaces from the different levels of theory against the CIS electronic surfaces.

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import rcParams

cqed_cis_spline = InterpolatedUnivariateSpline(r_array, np.real(cqed_cis_energy_array[2,:]), k=3)
cis_spline = InterpolatedUnivariateSpline(r_array, np.real(cis_energy_array[0,:]), k=3)


r_fine = np.linspace(1.85, 1.98, 100)
cqed_cis_e_fine = cqed_cis_spline(r_fine)
cis_e_fine = cis_spline(r_fine)


print("cqed_cis",r_fine[np.argmin(cqed_cis_e_fine)])
print("cis",r_fine[np.argmin(cqed_rhf_e_fine)])

# get curvatures of each surface
cqed_cis_f = cqed_cis_spline.derivative()
cqed_cis_c = cqed_cis_f.derivative()

cis_f = cis_spline.derivative()
cis_c = cis_f.derivative()


# get locations of equilibrium geometry
r_eq_cqed_cis = r_fine[np.argmin(cqed_cis_e_fine)]
r_eq_cis = r_fine[np.argmin(cis_e_fine)]

# print location of equilibrium bondlength
print("cqed_cis",r_eq_cqed_cis)
print("rhf",r_eq_cis)

# print curvature evaluated at equilibrium geometries
k_cqed_cis = cqed_cis_c(r_eq_cqed_cis)
k_cis = cis_c(r_eq_cis)

# print the force constant of each surface in atomic units
print("cqed_cis",k_cqed_cis)
print("cis",k_cis)


rcParams['font.family'] = 'serif'
rcParams['font.size'] = 12
plt.plot(r_fine, cqed_cis_e_fine, 'red')
plt.plot(r_fine, cis_e_fine, 'blue')
plt.plot(r_array, cis_energy_array[0,:], 'bo', label='A,0')
plt.plot(r_array, np.real(cqed_cis_energy_array[2,:]), 'go', label="CQED-CIS UP")
plt.xlabel("Bondlength (Angstroms)")
plt.ylabel("Energy (Hartrees)")
plt.legend()
plt.show()


# Deviations of CQED-CIS from models (Fig 6)
We will compare the lower and upper-polariton energies for MgH+ at a geometry of 2.2 Angstroms when coupled to a 4.75 eV photon with various coupling strength, looking for the onset of deviations between the eigenvalues from CQED-CIS and those coming from simpler model Hamiltonians parameterized by CIS values for lone MgH+.

In [ ]:
# template for z-matrix
mol_str = """
Mg
H 1 2.2
symmetry c1
1 1
"""
n_lam = 20
# electric field strength 
lam_z = np.linspace(0, 0.025, n_lam) 

# set psi4 options
psi4.set_options(options_dict)

# arrays to hold psi4 cis quantities to fit the model PF Hamiltonian with
# array for transition dipole moments
tdm_array = np.zeros((3, n_lam))

# array for total diplole of ground and excited state 
dipole_array = np.zeros((3,2,n_lam))

# array for ordinary cis
cis_energy_array = np.zeros((5,n_lam))

# array to hold the cs-cqed-cis energies
# array for cqed cis
cqed_cis_energy_array = np.zeros((5, n_lam),dtype=complex)

# array for model Pauli-Fierz energies
pf = np.zeros((3,n_lam),dtype=complex)

# array for Jaynes-Cummings energies
jc = np.zeros((2,n_lam),dtype=complex)


# photon energy
om_r = 4.75 / 27.211



print(" value of gamma to cross over from weak to strong coupling:", 4 * g_max)
om = om_r 

# lambda vector
#lam = np.array([0,0,lam_z])

# loop over the different bond-lengths, create different instances
# of MgH+ molecule and compute and store various quantities
ctr = 0
for l in lam_z:
    lam = np.array([0, 0, l])
    mol = psi4.geometry(mol_str)
    e, wfn = psi4.energy("scf/cc-pVDZ", return_wfn=True, molecule=mol)
    
    # confirm our cis agrees with psi4
    res = tdscf_excitations(wfn, states=3, triplets = "NONE", tda=True)
    
    # parse res for excitation energies
    psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]
    
    cis_e, cis_c, cis_mu, tdm = cis(mol_str, options_dict)
    
    cis_energy_array[0,ctr] = e
    cis_energy_array[0,ctr] = e+psi4_excitation_e[0]
    
    pf[0,ctr], pf[1,ctr], pf[2,ctr], vecs = pauli_fierz(e, e+cis_e[0], om, lam, cis_mu[:,0], cis_mu[:,1], tdm)
    jc[0,ctr], jc[1,ctr] = jaynes_cummings(e, e+cis_e[0], om, lam,  tdm)
    
    assert np.isclose(cis_e[0], psi4_excitation_e[0])
    assert np.isclose(cis_e[1], psi4_excitation_e[1])
    
    cqed_dict = cs_cqed_cis(lam, om, mol_str, options_dict)
    
    # get cqed-rhf energy
    cqed_scf_e = cqed_dict['CQED-RHF ENERGY']
    # get cqed-cis excitation energies
    cqed_cis_e = cqed_dict['CQED-CIS ENERGY']
    # store to arrays
    cqed_cis_energy_array[0:5,ctr] = cqed_scf_e+cqed_cis_e[0:5]
    ctr+=1
    

 

In [ ]:
plt.plot(lam_z, np.real(jc[0,:]), 'b--', label="JC LP")
plt.plot(lam_z, np.real(jc[1,:]), 'g--', label="JC UP")
plt.plot(lam_z, np.real(pf[1,:]), 'b-*', label="PF LP")
plt.plot(lam_z, np.real(pf[2,:]), 'g-*', label="PF UP")
plt.plot(lam_z, np.real(cqed_cis_energy_array[1,:]), 'bo', label="CQED-CIS LP")
plt.plot(lam_z, np.real(cqed_cis_energy_array[2,:]), 'go', label="CQED-CIS UP")
plt.legend()
plt.show()